# **🎯 Project Objective**

The objective of NeuralMatch AI is to design and implement a deep learning–powered Resume Screening and ATS Scoring System that leverages advanced neural network architectures, specifically Transformer-based sentence embeddings, to evaluate semantic alignment between a candidate’s resume and a job description.

This project emphasizes the application of modern deep neural networks in Natural Language Processing (NLP) to move beyond traditional keyword matching and enable contextual understanding of text.

The system aims to:

Utilize Transformer-based neural networks (Sentence-BERT) to generate high-dimensional semantic embeddings of resume and job description content.

Compute contextual similarity using cosine similarity in embedding space.

Integrate traditional NLP techniques (TF-IDF) with deep semantic representations for hybrid scoring.

Quantify recruiter-relevant signals such as action verbs, measurable impact, and keyword alignment.

Produce a weighted, recruiter-grade ATS score reflecting real-world hiring evaluation logic.

By combining deep representation learning with statistical text analysis, this project demonstrates how neural networks can model human-like understanding in recruitment automation systems.

# **🧠 Neural Network Emphasis (Core Intelligence of the System)**

At the heart of this system lies:

A Transformer-based Neural Network (all-MiniLM-L6-v2)

Built on the BERT architecture

Trained using contrastive learning for semantic similarity tasks

The neural network:

Converts sentences into dense vector embeddings

Captures contextual meaning rather than just word overlap

Understands relationships between concepts (e.g., “built ML model” ≈ “developed predictive system”)

Unlike rule-based ATS systems, this architecture models semantic intelligence using deep learning representations.

In [ ]:
!pip install pdfplumber sentence-transformers scikit-learn nltk

In [ ]:
import re
import pdfplumber
import numpy as np
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files

nltk.download('stopwords')

# ---------------------------------------------
# 1️⃣ Upload Resume
# ---------------------------------------------
print("Upload Resume (PDF)")
uploaded = files.upload()
resume_file = list(uploaded.keys())[0]

# ---------------------------------------------
# 2️⃣ Extract Resume Text
# ---------------------------------------------
def extract_text(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            content = page.extract_text()
            if content:
                text += content + " "
    return text

resume_raw = extract_text(resume_file)

# ---------------------------------------------
# 3️⃣ Clean Text
# ---------------------------------------------
def clean(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9% ]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

resume_text = clean(resume_raw)

jd_raw = input("\nPaste Job Description:\n")
jd_text = clean(jd_raw)

# ---------------------------------------------
# 4️⃣ Load Semantic Model
# ---------------------------------------------
print("\nLoading Model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model Loaded ✅")

# Sentence-Level Semantic Matching
resume_sentences = resume_raw.split(".")
jd_sentences = jd_raw.split(".")

resume_emb = model.encode(resume_sentences)

semantic_scores = []
for sentence in jd_sentences:
    jd_emb = model.encode([sentence])[0]
    sims = cosine_similarity([jd_emb], resume_emb)[0]
    semantic_scores.append(np.max(sims))

semantic_score = np.mean(semantic_scores)

# ---------------------------------------------
# 5️⃣ Dynamic Keyword Extraction (TF-IDF)
# ---------------------------------------------
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_features=100)
tfidf = vectorizer.fit_transform([resume_text, jd_text])

features = vectorizer.get_feature_names_out()

resume_vec = tfidf[0].toarray()[0]
jd_vec = tfidf[1].toarray()[0]

resume_keywords = set([features[i] for i in range(len(features)) if resume_vec[i] > 0])
jd_keywords = set([features[i] for i in range(len(features)) if jd_vec[i] > 0])

keyword_overlap = resume_keywords.intersection(jd_keywords)

keyword_score = len(keyword_overlap) / len(jd_keywords) if len(jd_keywords) > 0 else 0

# ---------------------------------------------
# 6️⃣ Business Verb Weighting
# ---------------------------------------------
business_verbs = [
    "led","delivered","optimized","improved","increased",
    "reduced","implemented","developed","built",
    "supported","drove","analyzed","designed","managed"
]

verb_hits = sum([1 for verb in business_verbs if verb in resume_text])
verb_score = min(verb_hits / len(business_verbs), 1)

# ---------------------------------------------
# 7️⃣ Impact Score (Numbers & % Detection)
# ---------------------------------------------
impact_patterns = re.findall(r'\d+%|\d+\s?percent|\d+\s?x', resume_text)
impact_score = min(len(impact_patterns) / 5, 1)

# ---------------------------------------------
# 8️⃣ JD Responsibility Boost
# ---------------------------------------------
important_jd_terms = [
    "responsibilities",
    "requirements",
    "role",
    "skills",
    "experience"
]

jd_weight = 1.2 if any(term in jd_text for term in important_jd_terms) else 1.0

# ---------------------------------------------
# 9️⃣ Final Recruiter-Grade Score
# ---------------------------------------------
final_score = (
    0.45 * semantic_score +
    0.30 * keyword_score +
    0.15 * verb_score +
    0.10 * impact_score
) * jd_weight

final_score = min(final_score, 1)

# ---------------------------------------------
# 🔟 Results
# ---------------------------------------------
print("\n==============================")
print(f"Semantic Score: {semantic_score*100:.2f}%")
print(f"Keyword Match: {keyword_score*100:.2f}%")
print(f"Business Verb Score: {verb_score*100:.2f}%")
print(f"Impact Score: {impact_score*100:.2f}%")
print(f"\n🔥 FINAL ATS SCORE: {final_score*100:.2f}%")
print("==============================")

print("\n✅ Matched Keywords:", list(keyword_overlap))
print("\n❌ Missing Keywords:", list(jd_keywords - resume_keywords))

# ---------------------------------------------
# Interpretation
# ---------------------------------------------
if final_score > 0.75:
    print("\n🟢 Recruiter-Level Strong Match")
elif final_score > 0.55:
    print("\n🟡 Moderate Match – Improve Alignment")
else:
    print("\n🔴 Needs Optimization")

Upload Resume (PDF)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving CV-Ramayan.pdf to CV-Ramayan.pdf

Paste Job Description:
We are seeking a detail-oriented and enthusiastic Fresher Data Analyst to join our dynamic team.  The ideal candidate will have a strong foundation in data analysis principles and a passion for transforming raw data into meaningful insights that drive business decisions.   Key Responsibilities:  Data Collection & Processing: Extract, clean, and organize data from various sources using tools like SQL and Excel.  Data Analysis: Perform exploratory data analysis to identify trends, patterns, and anomalies in datasets.  Data Visualization: Create clear and compelling visualizations using Tableau, Power BI, or Google Looker Studio to communicate findings.  Reporting: Develop and maintain reports and dashboards that support business operations and strategic planning.  Collaboration: Work closely with cross-functional teams to understand data needs and deliver actionable insights.  Continuous Learning: Stay updated on industry tr

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model Loaded ✅

Semantic Score: 62.64%
Keyword Match: 86.84%
Business Verb Score: 50.00%
Impact Score: 40.00%

🔥 FINAL ATS SCORE: 78.89%

✅ Matched Keywords: ['data', 'visualization', 'decisions', 'datasets', 'actionable', 'google', 'analyst', 'exploratory', 'excel', 'tools', 'team', 'collaboration', 'support', 'raw', 'detail', 'analysis', 'insights', 'business', 'bi', 'analytics', 'using', 'strong', 'dynamic', 'visualizations', 'trends', 'patterns', 'power', 'sql', 'clear', 'identify', 'reporting', 'tableau', 'transforming']

❌ Missing Keywords: ['candidate', 'closely', 'clean', 'communicate', 'compelling']

🟢 Recruiter-Level Strong Match


# **🏁 Conclusion**

The NeuralMatch AI project successfully demonstrates the practical application of deep neural networks and transformer-based language models in solving a real-world recruitment automation problem.

By leveraging Sentence Transformers built on advanced neural architectures, the system:

Achieves contextual understanding of resume and job descriptions

Moves beyond surface-level keyword matching

Quantifies candidate–role alignment using embedding similarity

Integrates deep learning with interpretable scoring mechanisms

This project highlights strong competency in:

Deep Learning

Transformer Architectures

NLP with Neural Networks

Semantic Embeddings

Hybrid AI Systems (Neural + Statistical Methods)

Ultimately, NeuralMatch AI showcases how modern neural networks can replicate aspects of human semantic reasoning, making recruitment evaluation more intelligent, scalable, and data-driven.